## Create a calibration using pyFAI

In [ ]:
import matplotlib.pyplot as plt
import pyFAI.gui.jupyter
import pyFAI
import fabio
from pyFAI.test.utilstest import UtilsTest
from pyFAI.calibrant import CALIBRANT_FACTORY
from pyFAI.goniometer import SingleGeometry
import pathlib
figure_size = 10

Load and show the calibration image

In [ ]:
filename = "calibration/DLS_CeO2_1200mm.tif"
frame = fabio.open(filename).data
figure = plt.figure(figsize=(figure_size, figure_size))
ax = plt.gca()
pyFAI.gui.jupyter.display(frame, ax=ax)

Define geometry components such as beam center and detector distance.

In [ ]:
x = 712.136 # x-coordinate of the beam-center in pixels
y = 727.864 # y-coordinate of the beam-center in pixels
d = 1194.046 # This is the distance in mm (unit used by Fit2d)
wl = 1.393000e-11 # Beam wavelength

Definition of the detector and of the calibrant

In [ ]:
detector = pyFAI.detectors.Detector(pixel1=0.000296, pixel2=0.000296)
print(detector)
calibrant = CALIBRANT_FACTORY("CeO2")
calibrant.wavelength = wl
calibrant

Set the geometry of the detector to be the guessed geometry

In [ ]:
initial_geometry = pyFAI.geometry.Geometry(detector=detector, wavelength=wl)
initial_geometry.setFit2D(d, x, y)
initial_geometry

We now use the SingleGeometry object to perform automatic ring extraction and calibration

In [ ]:
sg = SingleGeometry("demo", frame, calibrant=calibrant, detector=detector, geometry=initial_geometry)
sg.extract_cp(max_rings=10)

This figure shows that the fitted control points and rings overlap pretty. Refinement probably won't change much

In [ ]:
figure = plt.figure(figsize=(figure_size, figure_size))
ax = plt.gca()
pyFAI.gui.jupyter.display(sg=sg, ax=ax)

We refine the geometry, we leave the wavelength free as we know this but allow the other parameters to change. This accounts for any slight inaccuracy in the detector set up.

In [ ]:
sg.geometry_refinement.refine2(fix=["wavelength"])

This figure shows the result of the refinement.

In [ ]:
figure = plt.figure(figsize=(figure_size, figure_size))
ax = plt.gca()
pyFAI.gui.jupyter.display(sg=sg, ax=ax)

We can now save the refinement to a file to be used later

In [ ]:
# Delete the calibration if it already exists
calibration_path = "calibration/DLS_CeO2_1200mm_pyFAI.poni"
pathlib.Path(calibration_path).unlink(missing_ok=True)

#Save the geometry obtained
sg.geometry_refinement.save(calibration_path)
with open(calibration_path) as f:
    print(f.read())

We can see some very small changes in the rotation angles as a result of the refinement.

## Plot intensity changes over time

This notebook uses real data and creates an `Intensity-Time` plot for a single lattice plane.

Load calibration image.

In [ ]:
import math
import numpy as np

In [ ]:
number_of_cakes = 36
ai = pyFAI.load("calibration/DLS_CeO2_1200mm_pyFAI.poni")
# Rotate the detector so that the cardinal direction is in the center of the first cake.
cake_angle = 360 / number_of_cakes
ai.rot3 = (cake_angle / 2) * (math.pi / 180) # convert rotation to radians
print("\nIntegrator: \n", ai)

In [ ]:
def plot_single_slice(ai, input_file: str, two_theta_min: int, two_theta_max: int, 
                      number_of_points: int = 1000, number_of_cakes: int = 36000):
    
    """ Plot an azimuthal angle versus two-theta slice from a diffraction pattern image.
    
    :param input_file: input file name of the diffraction pattern image.
    :param two_theta_min: minimum two-theta value for the slicing of the data.
    :param two_theta_max: maximum two-theta value for the slicing of the data
    :param number_of_points: number of radial points in two-theta (default is 1000).
    :param number_of_cakes: number of azimuthal points in chi, default gives 0.01 degree resolution (default is 36000).
    
    :return: numpy array containing the azimuthal angles, and the intensity values for each time increment.
    """ 
    
    image = fabio.open(input_file).data
    
    result = ai.integrate2d(image, number_of_points, number_of_cakes, unit="2th_deg")
    
    result_intensity = result.intensity 
    result_radial = result.radial
    result_azimuthal = result.azimuthal
    
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)
    
    image1 = ax1.imshow(result_intensity,
               origin='lower',
               extent=[result_radial.min(), 
                       result_radial.max(), 
                       result_azimuthal.min(), 
                       result_azimuthal.max()],
               cmap='viridis',
               interpolation='none',
               aspect='auto',
               vmax = 100)
#     plt.colorbar(image1)
#     plt.xlabel(r"Scattering angle ${\theta}$ ($^{o}$)")   
#     plt.ylabel(r"Azimuthal angle ${\chi}$ ($^{o}$)");

    two_theta_min_value, two_theta_min_index = find_nearest(result_radial, two_theta_min)
    print("The closest two-theta value = ", two_theta_min_value, ", with index = ", two_theta_min_index)

    two_theta_max_value, two_theta_max_index = find_nearest(result_radial, two_theta_max)
    print("The closest two-theta value = ", two_theta_max_value, ", with index = ", two_theta_max_index)

    result_intensity_cropped = result_intensity[:,two_theta_min_index:two_theta_max_index]
    result_radial_cropped = result_radial[two_theta_min_index:two_theta_max_index]
    
    image2 = ax2.imshow(result_intensity_cropped,
               origin='lower',
               extent=[result_radial_cropped.min(), 
                       result_radial_cropped.max(), 
                       result_azimuthal.min(), 
                       result_azimuthal.max()],
               cmap='viridis',
               interpolation='none',
               aspect='auto',
               vmax = 100)
#     plt.colorbar(image2)
#     plt.xlabel(r"Scattering angle ${\theta}$ ($^{o}$)")   
#     plt.ylabel(r"Azimuthal angle ${\chi}$ ($^{o}$)");

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    index = (np.abs(array - value)).argmin()
    return array[index], index

In [ ]:
# user inputs
input_file = "data/MAUD/pixium_03100.tif"
two_theta_min = 3
two_theta_max = 4
number_of_points = 1000 # 10000 runs slow, so try 1000
number_of_cakes = 36000 # 36000 gives 0.01 degree resolution for azimuthal changes

plot_single_slice(ai, input_file, two_theta_min, two_theta_max, number_of_points, number_of_cakes)

Use the following function to save the intensity changes with time for a specific lattice plane.

In [ ]:
def get_intensity_time(ai, input_filepath: str, output_filepath:str, lattice_plane: str, two_theta_min: int, 
                       two_theta_max: int, number_of_points: int = 1000, number_of_cakes = 36000): 
    
    """ Get the intensity data around a ring for a given two-theta slice from a set of diffraction pattern 
    images, and save as a text file.
    
    :param ai: pyFAI detector calibration.
    :param input_filepath: input filepath containing the diffraction pattern images.
    :param output_filepath: output filepath for saving the analysis text file.
    :param lattice_plane: hkil or hkl indices of the lattice plane.
    :param two_theta_min: minimum two-theta value for the slicing of the data.
    :param two_theta_max: maximum two-theta value for the slicing of the data
    :param number_of_points: number of radial points in two-theta (default is 1000).
    :param number_of_cakes: number of azimuthal points in chi, default gives 0.01 degree resolution (default is 36000).
    
    :return: numpy array containing the azimuthal angles, and the intensity values for each time increment.
    """ 
    # Get a list of the files
    image_list = sorted(pathlib.Path(input_filepath).glob("pixium*"))

    start = True

    for image_path in tqdm(image_list):

        # create an image array and cake the data
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            image = fabio.open(image_path)
        pattern_image_array = image.data
        result2d = ai.integrate2d(pattern_image_array,
                                  number_of_points,
                                  number_of_cakes,
                                  unit="2th_deg",
                                  polarization_factor=0.99,
                                  method='full_csr')
        # get the caked results
        result_intensity = result2d.intensity 
        result_radial = result2d.radial
        result_azimuthal = result2d.azimuthal
        # Slice around a ring using the two-theta min and max values
        two_theta_min_value, two_theta_min_index = find_nearest(result_radial, two_theta_min)
        two_theta_max_value, two_theta_max_index = find_nearest(result_radial, two_theta_max)
        result_intensity_sum = result_intensity[:,two_theta_min_index:two_theta_max_index].sum(axis=1)

        if start:
            # create an empty array to store the results
            result_array = np.array([result_azimuthal,result_intensity_sum])
            start = False

        else:
            # stack the results in the array
            result_array = np.vstack((result_array,result_intensity_sum))

    # write out the data to a text file
    result_array = np.transpose(result_array)
    np.savetxt(f"{output_filepath}intensity_time_{lattice_plane}.txt", result_array)

    return (result_array)

In [ ]:
# user inputs
input_filepath = "data/MAUD/"
output_filepath = "analysis/"
lattice_plane = "(10-10)"
two_theta_min = 2
two_theta_max = 3
number_of_points = 1000 # 10000 runs slow, so try 1000
number_of_cakes = 36000 # 36000 gives 0.01 degree resolution for azimuthal changes

result_array = get_intensity_time(ai, input_filepath, output_filepath, lattice_plane, two_theta_min, 
                                  two_theta_max, number_of_points, number_of_cakes)

The result array contains the azimuthal values in the first column, and the intensity values with time, in the subsequent columns.

In [ ]:
intensity_array = result_array[:,1:]
azimuthal_array = result_array[:,0]
time = np.arange(0,np.size(intensity_array,1))

We can now plot the intensity changes for a single lattice plane with time.

In [ ]:
plt.imshow(intensity_array,
           origin='lower',
           extent=[time.min(), 
                   time.max(), 
                   azimuthal_array.min(), 
                   azimuthal_array.max()],
           cmap='viridis',
           interpolation='none',
           aspect='auto')

plt.colorbar()
plt.xlabel(r"Time (s)")   
plt.ylabel(r"Azimuthal angle ${\chi}$ ($^{o}$)");

## Plot intensity changes over time for different lattice planes